In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from subprocess import check_output
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.grid_search import GridSearchCV
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor,BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.linear_model import SGDRegressor,LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from catboost import CatBoostRegressor
from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer
import warnings
warnings.filterwarnings("ignore")
import re
from sklearn.decomposition import PCA
from sklearn import model_selection





train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
test_id=test.id


vector = np.vstack((train[['lattice_vector_1_ang', 'lattice_vector_2_ang','lattice_vector_3_ang']].values,
                    test[['lattice_vector_1_ang', 'lattice_vector_2_ang','lattice_vector_3_ang']].values))

pca = PCA().fit(vector)
train['vector_pca0'] = pca.transform(train[['lattice_vector_1_ang', 'lattice_vector_2_ang','lattice_vector_3_ang']])[:, 0]
test['vector_pca0'] = pca.transform(test[['lattice_vector_1_ang', 'lattice_vector_2_ang','lattice_vector_3_ang']])[:, 0]


X = train.drop(['id','bandgap_energy_ev','formation_energy_ev_natom'], axis=1)
Y_feen = train['formation_energy_ev_natom']
Y_bee = train['bandgap_energy_ev']

test = test.drop(['id'], axis = 1)


In [ ]:
def rmsle(h, y): 
    """
    Compute the Root Mean Squared Log Error for hypthesis h and targets y

    Args:
        h - numpy array containing predictions with shape (n_samples, n_targets)
        y - numpy array containing targets with shape (n_samples, n_targets)
    """
    return np.sqrt(np.square(np.log(h + 1) - np.log(y + 1)).mean())

rmsleS=make_scorer(rmsle)

In [ ]:
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(KFold(n_splits=self.n_splits, shuffle=True).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))
            print('********:',i)
            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                y_holdout = y[test_idx]
                print ("Fit %s fold %d" % (str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
                # cross_score = cross_val_score(clf, X_train, y_train, cv=5, scoring=rmsleS)
                # print("    cross_score: %.5f" % (cross_score.mean()))
                y_pred = clf.predict(X_holdout)               
                print('errorrmlse:',rmsle(y_pred,y_holdout))
                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict(T)
            S_test[:, i] = S_test_i.mean(axis=1)
        #print(S_train)
        results = cross_val_score(self.stacker, S_train, y, cv=5, scoring=rmsleS)
        print("Stacker score: %.5f" % (results.mean()))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict(S_test)
        return res

In [ ]:
# LightGBM params
lgb_params = {
    'num_leaves': 8,
    'objective': 'regression',
    'min_data_in_leaf': 18,
    'learning_rate': 0.04,
    'n_estimators':500,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'min_child_weight':35,
    'metric': 'l2',
    'max_depth':8,
    'reg_alpha':0.1,
    'reg_lambda':1,
    'seed':1000,
}


# RandomForest params
rf_params = {
    'max_features':'auto',
    'n_estimators':200,
    'n_jobs':1,
    'max_depth':12,
}

# rf_params['n_estimators'] = 200
# rf_params['max_depth'] = 6
# rf_params['min_samples_split'] = 70
# rf_params['min_samples_leaf'] = 30


# ExtraTrees params
et_params = {}
# et_params['n_estimators'] = 155
# et_params['max_features'] = 0.3
# et_params['max_depth'] = 6
# et_params['min_samples_split'] = 40
# et_params['min_samples_leaf'] = 18


# XGBoost params
xgb_params = {
    'silent':1,
    'learning_rate':0.01,
    'n_estimators':1000,
    'max_depth':4,
    'min_child_weight':6,
    'gamma':0,
    'subsample':0.7,
    'colsample_bylevel':1,
    'scale_pos_weight':1,
    'seed':27,
    'n_jobs':1,
    'reg_alpha':0.002,
}




# CatBoost params
cat_params = {
    'verbose':False,
    'iterations':1000
}

#cat_params['iterations'] = 900
# cat_params['depth'] = 8
# cat_params['rsm'] = 0.95
# cat_params['learning_rate'] = 0.03
# cat_params['l2_leaf_reg'] = 3.5
# cat_params['border_count'] = 8



#BaggingRegressor
bag_params={}




# Regularized Greedy Forest params
rgf_params = {}
# rgf_params['max_leaf'] = 2000
# rgf_params['learning_rate'] = 0.5
# rgf_params['algorithm'] = "RGF_Sib"
# rgf_params['test_interval'] = 100
# rgf_params['min_samples_leaf'] = 3
# rgf_params['reg_depth'] = 1.0
# rgf_params['l2'] = 0.5
# rgf_params['sl2'] = 0.005
lgb_model = LGBMRegressor(**lgb_params)

rf_model = RandomForestRegressor(**rf_params)

et_model = ExtraTreesRegressor(**et_params)

xgb_model = XGBRegressor(**xgb_params)



cat_model = CatBoostRegressor(**cat_params)

#rgf_model = RGFRegressor(**rgf_params)

gb_model = GradientBoostingRegressor(max_depth=5)


knn = KNeighborsRegressor()
ada_model = AdaBoostRegressor()
bag = BaggingRegressor()

ln = LinearRegression()

stack = Ensemble(n_splits=5,
        stacker = ln,
        base_models = (knn,gb_model,bag,cat_model,xgb_model,rf_model))

y_pred1 = stack.fit_predict(X, Y_feen, test)


######################################################################



In [ ]:
# LightGBM params
lgb2_params = {
        #'num_leaves': 10,
        'objective': 'regression',
        'min_data_in_leaf': 18,
        'learning_rate': 0.04,
        'n_estimators':100,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 1,
        'min_child_weight':4,
        'metric': 'l2',
        'max_depth':8,
        'reg_alpha':0.1,
        'reg_lambda':0.1,
        'seed':1000,
}

# RandomForest params
rf2_params = {
    'max_features':'auto',
    'n_estimators':200,
    'n_jobs':1,
    'max_depth':12,
}


# ExtraTrees params
et2_params = {}
# et_params['n_estimators'] = 155
# et_params['max_features'] = 0.3
# et_params['max_depth'] = 6
# et_params['min_samples_split'] = 40
# et_params['min_samples_leaf'] = 18


# XGBoost params
xgb2_params = {
    'silent':1,
    'learning_rate':0.1,
    'n_estimators':100,
    'max_depth':4,
    'min_child_weight':6,
    'gamma':0,
    'subsample':0.7,
    'colsample_bylevel':1,
    'scale_pos_weight':1,
    'seed':27,
    'n_jobs':1,
    'reg_alpha':0.002,
}





# CatBoost params
cat2_params = {
    'verbose':False,
    'iterations':1000
}



# BaggingRegressor
bag2_params = {}

# Regularized Greedy Forest params
rgf2_params = {}
# rgf_params['max_leaf'] = 2000
# rgf_params['learning_rate'] = 0.5
# rgf_params['algorithm'] = "RGF_Sib"
# rgf_params['test_interval'] = 100
# rgf_params['min_samples_leaf'] = 3
# rgf_params['reg_depth'] = 1.0
# rgf_params['l2'] = 0.5
# rgf_params['sl2'] = 0.005
lgb2_model = LGBMRegressor(**lgb2_params)

rf2_model = RandomForestRegressor(**rf2_params)

et2_model = ExtraTreesRegressor(**et2_params)

xgb2_model = XGBRegressor(**xgb2_params)

cat2_model = CatBoostRegressor(**cat2_params)

# rgf_model = RGFRegressor(**rgf_params)

gb2_model = GradientBoostingRegressor()


knn2 = KNeighborsRegressor()
ada2_model = AdaBoostRegressor()
bag2 = BaggingRegressor()

ln2 = LinearRegression()

stack2 = Ensemble(n_splits=5,
        stacker = ln2,
        base_models = (knn2,gb2_model,bag2,rf2_model,cat2_model,xgb2_model))

y_pred2 = stack2.fit_predict(X, Y_bee, test)

In [ ]:
sample = pd.read_csv("../input/sample_submission.csv")

sample["formation_energy_ev_natom"] = y_pred1
sample["bandgap_energy_ev"] = y_pred2

sample.to_csv("subtest.csv",index = False)